In [ ]:
from pathlib import Path

import geopandas as gpd
import quackosm as qosm
from lonboard import viz
from shapely import box, Point
from quackosm import osm_extracts

## How to find OSM data for a given region?

`QuackOSM` has logic to find matching `*.osm.pbf` extracts from existing sources.

It downloads information about available extracts with URLs and geometries and later tries to find smallest extracts that cover given region.

For example, `Geofabrik` has full `GeoJSON` index available on their website.

In [ ]:
GEOFABRIK_INDEX = "https://download.geofabrik.de/index-v1.json"
geofabrik_gdf = gpd.read_file(GEOFABRIK_INDEX)
geofabrik_gdf

In [ ]:
viz(geofabrik_gdf)

### Scenario 1: Simple polygon

#### Monaco

In [ ]:
geometry = box(
    xmin=7.415,
    xmax=7.425,
    ymin=43.727,
    ymax=43.735,
)
geometry

In [ ]:
qosm.convert_geometry_to_geodataframe(geometry)

What happens under the hood?

In [ ]:
matching_extracts = osm_extracts.find_smallest_containing_geofabrik_extracts(geometry=geometry)
matching_extracts

In [ ]:
viz([matching_extracts[0].geometry, geometry])

#### Andorra

In [ ]:
geometry = box(
    xmin=1.382,
    xmax=1.809,
    ymin=42.400,
    ymax=42.676,
)
geometry

In [ ]:
matching_extracts = osm_extracts.find_smallest_containing_geofabrik_extracts(geometry=geometry)
matching_extracts

In [ ]:
viz([*(extract.geometry for extract in matching_extracts), geometry])

### Scenario 2: Geocoded cities

In [ ]:
import osmnx as ox

geocoded_gdf = ox.geocode_to_gdf(["Berlin", "Paris", "Greater London", "Warsaw"])
geocoded_gdf

In [ ]:
matching_extracts = osm_extracts.find_smallest_containing_geofabrik_extracts(
    geometry=geocoded_gdf.unary_union
)
matching_extracts

In [ ]:
viz([*(extract.geometry for extract in matching_extracts), geocoded_gdf])

Change source from Geofabrik to BBBike

In [ ]:
matching_extracts = osm_extracts.find_smallest_containing_extract(
    geometry=geocoded_gdf.unary_union,
    source="BBBike",  # options: "Geofabrik", "BBBike", "osmfr", "any"
)
matching_extracts

In [ ]:
viz([*(extract.geometry for extract in matching_extracts), geocoded_gdf])

### Scenario 3: Null Island

In [ ]:
null_island = Point(0, 0)

qosm.convert_geometry_to_geodataframe(geometry_filter=null_island, osm_extract_source='BBBike')

Expand to polygon

In [ ]:
null_island_box = box(
    xmin=0,
    xmax=0.1,
    ymin=0,
    ymax=0.1,
)

qosm.convert_geometry_to_geodataframe(geometry_filter=null_island_box, osm_extract_source='BBBike')

Allow uncovered geometry

In [ ]:
qosm.convert_geometry_to_geodataframe(
    geometry_filter=null_island_box,
    osm_extract_source="BBBike",
    allow_uncovered_geometry=True,
)

### Scenario 4: Multiple cities across continent

In [ ]:
eurostat_cities_100k = gpd.read_file(
    Path(".").resolve().parent / "data/URAU_RG_100K_2021_4326_CITIES.geojson"
)
viz(eurostat_cities_100k)

In [ ]:
matching_extracts = osm_extracts.find_smallest_containing_geofabrik_extracts(
    geometry=eurostat_cities_100k.geometry.make_valid().unary_union
)
matching_extracts

In [ ]:
viz([*(extract.geometry for extract in matching_extracts), eurostat_cities_100k])

### Scenario 5: Mismatch between geocoder and extracts geometries

In [ ]:
monaco_nominatim_geometry = ox.geocode_to_gdf("Monaco")
monaco_nominatim_geometry

In [ ]:
matching_extracts = osm_extracts.find_smallest_containing_geofabrik_extracts(
    geometry=monaco_nominatim_geometry.unary_union
)
matching_extracts

In [ ]:
viz([matching_extracts[0].geometry, monaco_nominatim_geometry])

In [ ]:
monaco_geofabrik_extract_geometry = osm_extracts._get_geofabrik_index().query('id == "Geofabrik_monaco"')
monaco_geofabrik_extract_geometry

In [ ]:
viz([monaco_nominatim_geometry, monaco_geofabrik_extract_geometry])